In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nasdaqdatalink as ndlink
import yfinance as yf
from datetime import datetime

interval = (
    datetime(2013, 3, 1),
    datetime(2017, 5, 1)
)

index = pd.date_range(start=interval[0].strftime('%Y-%m-%d'), end=interval[1].strftime('%Y-%m-%d'), freq='D')

# OHLC Bitcoin

In [2]:
btc = pd.read_csv('bitcoin.csv')
btc['Date'] = pd.to_datetime(btc['Date'], format='%b %d, %Y')
btc.set_index('Date', inplace=True)
btc.sort_index(inplace=True)
btc.rename(columns={'Price': 'Close'}, inplace=True)
btc = btc[['Close', 'Open', 'High', 'Low']]
btc.head()

,Close,Open,High,Low
Date,,,,
2013-04-01,104.0,93.0,106.0,92.2
2013-04-02,118.0,104.0,118.4,99.0
2013-04-03,135.0,118.0,147.0,110.0
2013-04-04,132.1,135.0,142.1,116.4
2013-04-05,142.3,132.1,144.9,130.2


# Bitcoin Blockchain Data

In [3]:
authtoken = 'xx8e6Zzbphz_MLnS3wM_'

symbols = {
    'BCHAIN/TRFUS': 'Bitcoin Total Transaction Fees USD', #Bitcoin Total Transaction Fees USD
    'BCHAIN/TRVOU': 'Bitcoin USD Exchange Trade Volume', #Bitcoin USD Exchange Trade Volume
    'BCHAIN/HRATE': 'Bitcoin Hash Rate', #Bitcoin Hash Rate
    'BCHAIN/CPTRA': 'Bitcoin Cost Per Transaction', #Bitcoin Cost Per Transaction
    'BCHAIN/NTRAN': 'Bitcoin Number of Transactions', #Bitcoin Number of Transactions
}

data = {}
for symbol in symbols.keys():
    query = ndlink.get(symbol, authtoken=authtoken)
    query = query.loc[interval[0].strftime('%Y-%m-%d'):interval[1].strftime('%Y-%m-%d'),:]
    data[symbols[symbol]] = query

blockchain = pd.concat(list(data.values()), axis=1)
blockchain.columns = list(data.keys())
blockchain = blockchain.reindex(index)
blockchain.head()

,Bitcoin Total Transaction Fees USD,Bitcoin USD Exchange Trade Volume,Bitcoin Hash Rate,Bitcoin Cost Per Transaction,Bitcoin Number of Transactions
2013-03-01,1862.939394,4.230278e+06,34.306270,2.143520,60659.0
2013-03-02,1413.137316,3.877987e+05,33.003501,2.639044,49631.0
2013-03-03,1570.901496,1.487783e+06,37.997451,2.602752,54459.0
2013-03-04,2048.096778,4.294412e+05,37.997451,2.715824,56039.0
2013-03-05,2878.072750,5.075535e+06,35.174784,2.938498,66437.0


# External Data

In [4]:
external_symbols = {
    'CL=F': 'Crude Oil',
    'GC=F': 'Gold',
    'NQ=F': 'NASDAQ Future',
    'ES=F': 'S&P500 Future',
    '^GDAXI': 'DAX Index'
}

data = {}

ts = yf.download(
    ' '.join(list(external_symbols.keys())),
    start=interval[0].strftime('%Y-%m-%d'),
    end=interval[1].strftime('%Y-%m-%d'),
    threads=True,
)

ts = ts.reindex(index)
external = ts.loc[:, 'Close'].copy()
external.rename(columns=external_symbols, inplace=True)
external.head()

[*********************100%***********************]  5 of 5 completed


,Crude Oil,S&P500 Future,Gold,NASDAQ Future,DAX Index
2013-03-01,90.680000,1516.50,1571.900024,2748.75,7708.160156
2013-03-02,NaN,NaN,NaN,NaN,NaN
2013-03-03,NaN,NaN,NaN,NaN,NaN
2013-03-04,90.120003,1525.75,1572.099976,2760.00,7691.680176
2013-03-05,90.820000,1537.00,1574.599976,2798.25,7870.310059


In [5]:
dataset = pd.concat((btc, blockchain, external), axis=1)
dataset = dataset.reindex(index)
dataset.fillna(method='ffill', inplace=True)
dataset = dataset.loc['2013-04-01':'2017-04-01', :]
dataset.head()

,Close,Open,High,Low,Bitcoin Total Transaction Fees USD,Bitcoin USD Exchange Trade Volume,Bitcoin Hash Rate,Bitcoin Cost Per Transaction,Bitcoin Number of Transactions,Crude Oil,S&P500 Future,Gold,NASDAQ Future,DAX Index
2013-04-01,104.0,93.0,106.0,92.2,6264.561480,8.000595e+06,58.248983,7.995808,60268.0,97.070000,1556.0,1600.000000,2788.00,7795.310059
2013-04-02,118.0,104.0,118.4,99.0,8699.607749,1.788011e+07,59.247537,7.909067,63276.0,97.190002,1564.5,1575.099976,2812.75,7943.870117
2013-04-03,135.0,118.0,147.0,110.0,10510.402465,9.529646e+06,59.247537,7.797820,69048.0,94.449997,1548.5,1552.800049,2787.00,7874.750000
2013-04-04,132.1,135.0,142.1,116.4,7572.555749,9.283812e+06,54.925473,9.367276,56063.0,93.260002,1554.5,1551.800049,2785.00,7817.390137
2013-04-05,142.3,132.1,144.9,130.2,7954.781556,4.643979e+06,64.079718,9.283940,61373.0,92.699997,1546.0,1575.400024,2763.00,7658.750000


In [6]:
dataset.isnull().sum()

Close                                 0
Open                                  0
High                                  0
Low                                   0
Bitcoin Total Transaction Fees USD    0
Bitcoin USD Exchange Trade Volume     0
Bitcoin Hash Rate                     0
Bitcoin Cost Per Transaction          0
Bitcoin Number of Transactions        0
Crude Oil                             0
S&P500 Future                         0
Gold                                  0
NASDAQ Future                         0
DAX Index                             0
dtype: int64

In [7]:
dataset.to_csv('dataset.csv')